In [14]:
import pandas as pd

df = pd.read_csv('../data/sample_products.csv')

df.head()

,product_id,name,category,brand,price,tags,description
0,1,PageTurner Electronics #1,Electronics,PageTurner,439.23,battery|wireless|bluetooth|4K,PageTurner electronics item designed for batte...
1,2,PageTurner Beauty #2,Beauty,PageTurner,292.59,anti-aging|SPF|vitamin-c|sensitive|skincare,PageTurner beauty item designed for anti-aging...
2,3,PageTurner Fashion #3,Fashion,PageTurner,373.81,summer|formal|casual|leather,PageTurner fashion item designed for summer an...
3,4,Silk&Co Home #4,Home,Silk&Co,402.76,eco-friendly|durable|minimal,Silk&Co home item designed for eco-friendly an...
4,5,PageTurner Beauty #5,Beauty,PageTurner,337.18,SPF|hydrating|anti-aging|skincare|vitamin-c,PageTurner beauty item designed for SPF and hy...


In [16]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['category'] = encoder.fit_transform(df['category'])

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming 'product_name' is a text feature
vectorizer = TfidfVectorizer(max_features=100)
X = vectorizer.fit_transform(df['name'])
